
<a href="https://colab.research.google.com/github/takzen/pytorch-black-belt/blob/main/17_Samplers_and_Imbalance.ipynb" target="_parent">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


In [ ]:
# --------------------------------------------------------------
# ☁️ COLAB SETUP (Automatyczna instalacja środowiska)
# --------------------------------------------------------------
import sys
import os

# Sprawdzamy, czy jesteśmy w Google Colab
if 'google.colab' in sys.modules:
    print('☁️ Wykryto środowisko Google Colab. Konfiguruję...')

    # 1. Pobieramy plik requirements.txt bezpośrednio z repozytorium
    !wget -q https://raw.githubusercontent.com/takzen/ai-engineering-handbook/main/requirements.txt -O requirements.txt

    # 2. Instalujemy biblioteki
    print('⏳ Instaluję zależności (to może chwilę potrwać)...')
    !pip install -q -r requirements.txt

    print('✅ Gotowe! Środowisko jest zgodne z repozytorium.')
else:
    print('💻 Wykryto środowisko lokalne. Zakładam, że masz już uv/venv.')


# 🥋 Lekcja 17: Samplers & Imbalance (Walka z Nierównowagą)

W domyślnym `DataLoader(shuffle=True)` każda próbka ma takie samo prawdopodobieństwo wyboru.
Przy niezbalansowanych danych (Imbalanced Data) to katastrofa.

**WeightedRandomSampler** działa jak ruletka, gdzie pola mają różne rozmiary.
1.  Liczymy wagi dla każdej klasy (odwrotnie proporcjonalne do liczebności).
2.  Przypisujemy wagę do każdej próbki w zbiorze.
3.  Sampler losuje indeksy na podstawie tych wag.

Efekt? Mimo że w danych masz 90% klasy A i 10% klasy B, w Batchu zobaczysz 50% A i 50% B.

In [1]:
import torch
from torch.utils.data import DataLoader, TensorDataset, WeightedRandomSampler

# 1. GENERUJEMY NIEZBALANSOWANE DANE
# 90 zer (Klasa większościowa)
# 10 jedynek (Klasa mniejszościowa - Rzadka)
labels = torch.cat([torch.zeros(90), torch.ones(10)]).long()
data = torch.randn(100, 5) # Jakieś losowe cechy

dataset = TensorDataset(data, labels)

print(f"Liczba próbek: {len(labels)}")
print(f"Liczba zer: {(labels == 0).sum()}")
print(f"Liczba jedynek: {(labels == 1).sum()}")
print("Proporcja: 9:1")

Liczba próbek: 100
Liczba zer: 90
Liczba jedynek: 10
Proporcja: 9:1


## Problem: Standardowy Loader

Zobaczmy, co się stanie, gdy użyjemy zwykłego Loadera.
W batchu o rozmiarze 10 spodziewamy się średnio jednej jedynki (albo zera). Model prawie nigdy nie zobaczy klasy mniejszościowej.

In [2]:
# Zwykły loader
loader_imbalanced = DataLoader(dataset, batch_size=10, shuffle=True)

print("--- ZWYKŁY LOADER ---")
for i, (x, y) in enumerate(loader_imbalanced):
    print(f"Batch {i}: {y.tolist()}")
    if i >= 2: break # Pokaż tylko 3 pierwsze

print("\nWniosek: Widzisz prawie same zera. Model uzna, że jedynki to błąd statystyczny.")

--- ZWYKŁY LOADER ---
Batch 0: [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
Batch 1: [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
Batch 2: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

Wniosek: Widzisz prawie same zera. Model uzna, że jedynki to błąd statystyczny.


## Rozwiązanie: Obliczanie Wag

Musimy nadać wagę każdej próbce.
Zasada: **Im rzadsza klasa, tym większa waga.**

Wzór: $W_{class} = \frac{1}{\text{Liczba próbek w tej klasie}}$

*   Waga dla 0: $1/90 \approx 0.011$
*   Waga dla 1: $1/10 = 0.1$ (ok. 9x większa!)

In [3]:
# 1. Liczymy wystąpienia klas
class_counts = [90, 10] # [Zera, Jedynki]
num_samples = sum(class_counts)

# 2. Liczymy wagi dla klas (1 / count)
class_weights = [1.0 / c for c in class_counts]

# 3. Przypisujemy wagę do KAŻDEJ PRÓBKI w zbiorze
# Tworzymy listę 100 wag. Jeśli próbka to 0 -> waga mała. Jeśli 1 -> waga duża.
sample_weights = [class_weights[label] for label in labels]

# Zamieniamy na tensor
sample_weights = torch.DoubleTensor(sample_weights)

print(f"Waga dla klasy 0: {class_weights[0]:.4f}")
print(f"Waga dla klasy 1: {class_weights[1]:.4f}")
print(f"Przykładowe wagi próbek: {sample_weights[:5]} ... {sample_weights[-5:]}")

Waga dla klasy 0: 0.0111
Waga dla klasy 1: 0.1000
Przykładowe wagi próbek: tensor([0.0111, 0.0111, 0.0111, 0.0111, 0.0111], dtype=torch.float64) ... tensor([0.1000, 0.1000, 0.1000, 0.1000, 0.1000], dtype=torch.float64)


## Użycie WeightedRandomSampler

Tworzymy sampler i przekazujemy go do `DataLoader`.

**Ważne:**
1.  `replacement=True`: Pozwala wylosować tę samą próbkę (tę rzadką jedynkę) wiele razy w jednej epoce. To klucz do oversamplingu.
2.  `shuffle=False`: W Loaderze musimy wyłączyć shuffle, bo Sampler i tak losuje (te dwie opcje się wykluczają).

In [4]:
# Tworzymy Sampler
# num_samples=len(sample_weights) oznacza, że w epoce chcemy zobaczyć 100 próbek (tyle co oryginał),
# ale będą one sztucznie zbalansowane.
sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)

# Tworzymy Loader z Samplerem (shuffle musi być False!)
loader_balanced = DataLoader(dataset, batch_size=10, sampler=sampler)

print("--- ZBALANSOWANY LOADER ---")
total_zeros = 0
total_ones = 0

for i, (x, y) in enumerate(loader_balanced):
    print(f"Batch {i}: {y.tolist()}")
    total_zeros += (y == 0).sum().item()
    total_ones += (y == 1).sum().item()

print("-" * 30)
print(f"Suma Zer: {total_zeros}")
print(f"Suma Jedynek: {total_ones}")
print("Widzisz? Mimo że mamy tylko 10 jedynek w bazie, loader podał ich około 50!")

--- ZBALANSOWANY LOADER ---
Batch 0: [1, 1, 1, 0, 0, 0, 1, 0, 1, 0]
Batch 1: [0, 0, 1, 0, 1, 1, 0, 0, 0, 1]
Batch 2: [0, 1, 1, 0, 1, 0, 1, 1, 0, 0]
Batch 3: [1, 0, 0, 0, 0, 1, 0, 0, 0, 0]
Batch 4: [1, 0, 0, 0, 0, 0, 0, 0, 1, 1]
Batch 5: [0, 0, 0, 0, 0, 1, 0, 1, 0, 0]
Batch 6: [1, 1, 0, 0, 1, 1, 1, 0, 1, 1]
Batch 7: [0, 1, 1, 0, 1, 1, 1, 1, 1, 0]
Batch 8: [1, 0, 1, 1, 0, 0, 1, 0, 1, 0]
Batch 9: [0, 1, 1, 1, 1, 1, 0, 0, 1, 0]
------------------------------
Suma Zer: 54
Suma Jedynek: 46
Widzisz? Mimo że mamy tylko 10 jedynek w bazie, loader podał ich około 50!


## 🥋 Black Belt Summary

1.  **Nie modyfikuj danych na dysku.** To strata miejsca. Używaj Samplera.
2.  **`replacement=True`**: To jest magia. Dzięki temu Sampler "klonuje" rzadkie przypadki w locie.
3.  **Wzór na wagi:** Zawsze `1 / count`.
4.  **Pułapka:** Nie używaj `shuffle=True` razem z `sampler`. PyTorch rzuci błędem.

W następnej lekcji zejdziemy najniżej jak się da w inżynierii danych: **Multiprocessing i Pin Memory**. Zrozumiemy, dlaczego Twój procesor (CPU) dławi kartę graficzną.